In [1]:
import openpyxl
import nltk
import string
import re
import pandas as pd
import numpy as np

from scipy import stats
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize

# NLP pre-processing

## Original data

In [43]:
# df = pd.read_excel("./data/dataset_balanced_4000.xlsx")
# df = pd.read_excel("./data/dataset_unbalanced_4000.xlsx")
df = pd.read_excel("./data/dataset_balanced_8000.xlsx")

df.head()

,ID,review,label,new_label,remove_all_stopwords,remove_some_stopwords,stemming,lemmatization
0,1,Not able to add freinds. It show something wen...,bug report,0,able add freinds show something went wrong rec...,not able add freinds show something went wrong...,not abl to add freind it show someth went wron...,not able to add freinds it show something went...
1,2,Calls keep dropping for no reason and is super...,bug report,0,calls keep dropping reason super annoying,calls keep dropping no reason super annoying,call keep drop for no reason and is super annoy,call keep dropping for no reason and is super ...
2,3,Can't access to my account solve this issue im...,bug report,0,cant access account solve issue immediately,cant access account solve issue immediately,cant access to my account solv thi issu immedi,cant access to my account solve this issue imm...
3,4,I don't know what's wrong with my own WhatsApp...,bug report,0,dont know whats wrong whatsapp working want si...,dont know whats wrong whatsapp not working wan...,i dont know what wrong with my own whatsapp it...,i dont know whats wrong with my own whatsapp i...
4,5,Cannot record audio while taking video,bug report,0,cannot record audio taking video,cannot record audio taking video,cannot record audio while take video,cannot record audio while taking video


In [42]:
df['label'].value_counts()

bug report         2000
rating             2000
feature request    2000
user experience    2000
Name: label, dtype: int64

In [8]:
# workbook = openpyxl.load_workbook("./data/dataset_NLU.xlsx")
# sheet = workbook['Sheet1']

# max_row = sheet.max_row
# for i in range(2,max_row+1):
#     label = sheet.cell(i, 3).value
#     if label == 'bug report':
#         sheet.cell(i, 4).value = 0
#     if label == 'feature request':
#         sheet.cell(i, 4).value = 1
#     if label == 'rating':
#         sheet.cell(i, 4).value = 2
#     if label == 'user experience':
#         sheet.cell(i, 4).value = 3
    
# workbook.save("./data/dataset_NLU.xlsx")
# workbook.close()
# print("over")

## 1. Remove Full stop words

In [29]:
en_stops = set(stopwords.words('english'))

def removeFullstop(review: str) -> list:
    # remove non alphanumeric 
    data_clean = "".join(re.findall(r'[A-Za-z0-9 ]', review))
    # case folding
    lower_data = data_clean.lower()
    # split into list
    data_list = lower_data.split(" ")
    # remove blank space
    data_list = [i for i in data_list if i != ""]
    # remove stopwords
    token = [i for i in data_list if i not in en_stops]
    
    return token

In [30]:
workbook = openpyxl.load_workbook("./data/dataset_balanced_8000.xlsx")
# workbook = openpyxl.load_workbook("./data/dataset_unbalanced_4000.xlsx")
sheet = workbook['Sheet1']

review = sheet.cell(2, 2).value
token = removeFullstop(review)
print(token)

['able', 'add', 'freinds', 'show', 'something', 'went', 'wrong', 'recommend', 'install']


In [31]:
max_row = sheet.max_row
for i in range(2,max_row+1):
    review = sheet.cell(i, 2).value
    word = removeFullstop(review)
    sheet.cell(i, 5).value = str(" ".join(word))

workbook.save("./data/dataset_balanced_8000.xlsx")
# workbook.save("./data/dataset_unbalanced_4000.xlsx")
workbook.close()
print("over")

over


## 2. Remove Some stop words

In [32]:
en_stops = set(stopwords.words('english'))
print(en_stops)

{'ain', "you're", 'them', 'been', 'again', 'off', 'those', 'under', 'down', "weren't", 'these', 'nor', 'your', 've', 'had', 'each', 'from', 'needn', "wasn't", 'she', 'me', 'ourselves', 'it', 'did', 'yourself', 't', 'myself', 'if', 'both', 'will', 'until', 'very', 'our', 'after', "should've", 's', 'but', "hasn't", 'up', 'haven', 'during', 'once', 'between', "don't", "couldn't", "you've", "she's", 'does', "isn't", "you'll", "you'd", 'above', 'as', 'was', "aren't", 'his', 'and', 'by', "mightn't", 'they', 'being', 'no', "mustn't", 'themselves', 'hasn', 'further', 'we', 'shouldn', 'hadn', 'own', 'my', 'herself', 'yourselves', 'him', "didn't", 'wasn', "that'll", 'are', 'be', 'he', 're', 'in', 'out', 'than', 'now', "doesn't", 'itself', 'yours', 'doing', 'what', 'why', 'most', 'there', 'aren', 'how', 'can', 'theirs', 'its', 'isn', 'with', 'won', 'some', 'any', "hadn't", 'wouldn', 'about', 'ours', 'm', 'mightn', 'before', 'that', 'or', 'should', "haven't", 'do', 'which', 'where', 'such', "needn

In [33]:
some_en_stops = {'each', 'does', 'by', 'of', 'o', 'between', 'herself', 'been', 'both', 'under', "she's", 'ourselves', 'what', 'our', 'was', 'such', 'an', 'if', 'before', 'own', 'your', 'who', 'her', 'him', 'we', 'its', 'yourself', 'theirs', 'or', 'how', 'here', 'with', 'for', 'themselves', 'any', 'them', 'those', 'yours', 'himself', 'doing', 'from', 'am', 'd', 'in', 'a', 'did', 'and', 're', 'myself', 'at', 'she', "you're", 'some', 'his', 'do', 'my', 'into', 'just', 'ours', 'is', 'above', 'over', 've', 'when', 'these', 'once', 'other', 'why', 'their', 'ma', 'about', 'y', "that'll", 'hasn', 'it', 'are', 'were', 'has', 'yourselves', "doesn't", 'shan', 'that', 'ain', 'they', 'i', 'whom', 'weren', "it's", 'itself', 'there', 'while', 'you', 'he', 'then', 'm', 'the', 'this', 'as', 'be', 'on', 'having', 'had', 'which', "you've", 'to', 'where', 'me', 'being', 's'}

In [34]:
def removeSomestop(review: str) -> list:
    # remove non alphanumeric 
    data_clean = "".join(re.findall(r'[A-Za-z0-9 ]', review))
    # case folding
    lower_data = data_clean.lower()
    # split into list
    data_list = lower_data.split(" ")
    # remove blank space
    data_list = [i for i in data_list if i != ""]
    # remove stopwords
    token = [i for i in data_list if i not in some_en_stops]
    
    return token

In [35]:
workbook = openpyxl.load_workbook("./data/dataset_balanced_8000.xlsx")
# workbook = openpyxl.load_workbook("./data/dataset_unbalanced_4000.xlsx")
sheet = workbook['Sheet1']

review = sheet.cell(2, 2).value
token = removeSomestop(review)
print(token)

['not', 'able', 'add', 'freinds', 'show', 'something', 'went', 'wrong', 'recommend', 'not', 'install']


In [36]:
max_row = sheet.max_row
for i in range(2,max_row+1):
    review = sheet.cell(i, 2).value
    word = removeSomestop(review)
    sheet.cell(i, 6).value = str(" ".join(word))

workbook.save("./data/dataset_balanced_8000.xlsx")
# workbook.save("./data/dataset_unbalanced_4000.xlsx")
workbook.close()
print("over")

over


## 3. Stemming

In [17]:
porter = nltk.PorterStemmer()

def stemming(review: str) -> list:
    # remove non alphanumeric 
    data_clean = "".join(re.findall(r'[A-Za-z0-9 ]', review))
    # case folding
    lower_data = data_clean.lower()
    # split into list
    data_list = lower_data.split(" ")
    # remove blank space
    data_list = [i for i in data_list if i != ""]
    # Stemming
    token = [porter.stem(w) for w in data_list]
    
    return token

In [37]:
workbook = openpyxl.load_workbook("./data/dataset_balanced_8000.xlsx")
# workbook = openpyxl.load_workbook("./data/dataset_unbalanced_4000.xlsx")
sheet = workbook['Sheet1']

review = sheet.cell(2, 2).value
token = stemming(review)
print(token)

['not', 'abl', 'to', 'add', 'freind', 'it', 'show', 'someth', 'went', 'wrong', 'recommend', 'not', 'to', 'instal', 'it']


In [38]:
max_row = sheet.max_row
for i in range(2,max_row+1):
    review = sheet.cell(i, 2).value
    word = stemming(review)
    sheet.cell(i, 7).value = str(" ".join(word))

workbook.save("./data/dataset_balanced_8000.xlsx")
# workbook.save("./data/dataset_unbalanced_4000.xlsx")
workbook.close()
print("over")

over


## Lemmatization

In [21]:
wnl = nltk.WordNetLemmatizer()

def lemmatization(review: str) -> list:
    # remove non alphanumeric 
    data_clean = "".join(re.findall(r'[A-Za-z0-9 ]', review))
    # case folding
    lower_data = data_clean.lower()
    # split into list
    data_list = lower_data.split(" ")
    # remove blank space
    data_list = [i for i in data_list if i != ""]
    # Lemmatization
    token = [wnl.lemmatize(s) for s in data_list]
    
    return token

In [39]:
workbook = openpyxl.load_workbook("./data/dataset_balanced_8000.xlsx")
# workbook = openpyxl.load_workbook("./data/dataset_unbalanced_4000.xlsx")
sheet = workbook['Sheet1']

review = sheet.cell(2, 2).value
print(review)
token = lemmatization(review)
print(token)

Not able to add freinds. It show something went wrong.. Recommend not to install it
['not', 'able', 'to', 'add', 'freinds', 'it', 'show', 'something', 'went', 'wrong', 'recommend', 'not', 'to', 'install', 'it']


In [40]:
max_row = sheet.max_row
for i in range(2,max_row+1):
    review = sheet.cell(i, 2).value
    word = lemmatization(review)
    sheet.cell(i, 8).value = str(" ".join(word))

workbook.save("./data/dataset_balanced_8000.xlsx")
# workbook.save("./data/dataset_unbalanced_4000.xlsx")
workbook.close()
print("over")

over


In [20]:
en_stops = set(stopwords.words('english'))
# wnl = nltk.WordNetLemmatizer()
porter = nltk.PorterStemmer()

def ReviewProcessing(review: str) -> list:
    # remove non alphanumeric 
    data_clean = "".join(re.findall(r'[A-Za-z0-9 ]', review))
    # case folding
    lower_data = data_clean.lower()
    # split into list
    data_list = lower_data.split(" ")
    # remove blank space
    data_list = [i for i in data_list if i != ""]
    # remove stopwords
    # word = [i for i in data_list if i not in en_stops]
    # Stemming
    word_stem = [porter.stem(w) for w in word]
#     # Lemmatization
#     data = [wnl.lemmatize(s) for s in word_stem]
    
    return word

In [9]:
review = sheet.cell(3907, 2).value
word = ReviewProcessing(review)
print(word)

['trying', 'get', 'take', 'card', 'one', 'person', 'contact', 'way', 'want', 'anything', 'company', 'cant', 'old', 'fashion', 'talk', 'someone', 'sorry', 'company', 'wish', 'deal', 'youre', 'u', 'cant', 'speak']


In [10]:
max_row = sheet.max_row
for i in range(2,max_row+1):
    review = sheet.cell(i, 2).value
    word = ReviewProcessing(review)
    sheet.cell(i, 5).value = str(" ".join(word))

workbook.save("./data/dataset.xlsx")
workbook.close()
print("over")

over


In [43]:
frame = pd.read_excel("./data/dataset_NLU.xlsx")
df_reviews_content = pd.DataFrame(frame, columns=["preprocessing","label"])
df_reviews_content.head()

,preprocessing,label
0,able add freinds show something went wrong .. ...,bug report
1,calls keep dropping reason super annoying,bug report
2,ca n't access account solve issue immediately,bug report
3,n't know 's wrong whatsapp working want sign b...,bug report
4,record audio taking video,bug report
5,ca n't turn notifications app crush notificati...,bug report
6,videos turning green shoot .. please help,bug report
7,snaps n't sending reason time snapchats wrong ...,bug report
8,app disappointing messages taking long send,bug report
9,try make filter video 's hanging video lagging...,bug report
